In [15]:
import json

import nltk
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# Suppress warnings
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

In [2]:
class Schema:
    def __init__ (self, id):
        self.id = id
        self.tableNames = []
        self.columnNames = []
        self.columnAtributes = []

In [3]:
stopWordSet = nltk.corpus.stopwords.words('english')
stemmer = nltk.SnowballStemmer(language='english')
tokenizer = nltk.RegexpTokenizer(r"[a-zA-Z0-9]+")

# Data Analysis and Preprocessing

We note that the databases with the largest amounts of queries in Spider are college_2/college_1, hr_1, store_1, and soccer_2.
We will create 4 models applied on each of these databases.

In [4]:
# Loading Spider Data
with open('./spider-data/tables.json', 'r') as f:
  schemaDoc = json.load(f)

with open('./spider-data/train_spider.json', 'r') as f:
  queryDoc = json.load(f)

schemas = {}
db_counts = {}

for idx, iquery in enumerate(queryDoc):
    db_counts[iquery['db_id']] = db_counts.get(iquery['db_id'], 0) + 1

print(sorted(db_counts.items(), key=lambda x:x[1], reverse=True))

[('college_2', 170), ('college_1', 164), ('hr_1', 124), ('store_1', 112), ('soccer_2', 106), ('bike_1', 104), ('music_1', 100), ('hospital_1', 100), ('music_2', 100), ('dorm_1', 100), ('allergy_1', 98), ('movie_1', 98), ('flight_1', 96), ('driving_school', 93), ('cre_Doc_Tracking_DB', 90), ('department_store', 88), ('customers_and_addresses', 88), ('activity_1', 88), ('network_2', 86), ('products_gen_characteristics', 86), ('game_1', 86), ('chinook_1', 84), ('cre_Theme_park', 84), ('cre_Docs_and_Epenses', 84), ('customers_and_invoices', 82), ('sakila_1', 82), ('baseball_1', 82), ('e_learning', 82), ('cre_Drama_Workshop_Groups', 82), ('wine_1', 82), ('flight_4', 82), ('customers_card_transactions', 80), ('apartment_rentals', 80), ('formula_1', 80), ('loan_1', 80), ('manufactory_1', 80), ('tracking_grants_for_research', 78), ('inn_1', 74), ('college_3', 74), ('voter_2', 72), ('csu_1', 70), ('club_1', 70), ('election', 68), ('student_1', 68), ('icfp_1', 66), ('music_4', 60), ('tracking_or

### Create outputs

We first filter the query and schema doc entries to the queries linked to the DB labeled by `schema`.
In our application, it will be `college_2`, `hr_1`, `store_1`, and `soccer_2`.

Then we create our outputs (y) for the model. The outputs are in the following format:

e.g. `[0,0,0,1,0,1,1,...]`

Where the *nth* index corresponds to the *nth* table in the schema. The value contains a 1 if the table is referenced in the SQL query, and a 0 if it is not.

In [5]:
def getEmbeddings(schemaDoc, queryDoc, schemaName):

    filteredQueryDoc = [query for query in queryDoc if query['db_id'] == schemaName]
    filteredSchemaDoc = [schema for schema in schemaDoc if schema['db_id'] == schemaName]

    for idx, iSchema in enumerate(filteredSchemaDoc):
        dbid = iSchema['db_id']
        test = Schema(dbid)
        for columnName in iSchema['table_names_original']:
            test.tableNames.append(columnName)
        for columnName in iSchema['column_names_original']:
            test.columnNames.append(columnName[1])
            test.columnAtributes.append(columnName[0])
        # print("Schema", idx, ":", test.id, ":", test.tableNames)
        schemas[dbid] = test

    tables = []
    columns = []

    for idx, iquery in enumerate(filteredQueryDoc):
        # print(schemas)
        # Initialize a blank array with the length of the number of tables
        tablesArray = [0] * len(schemas[iquery['db_id']].tableNames)
        columnsArray = [0] * len(schemas[iquery['db_id']].columnNames)
        # Get all the tokens from teh particular query we are looking at
        queryTokens = iquery['query_toks_no_value']

        # Search through all the tokens in the given query
        for idx, queryToken in enumerate(queryTokens):
            # Only look at tokens that come after the word "from," since those are the table names
            if (idx > 0) & (queryTokens[idx - 1].lower() == "from") | (queryTokens[idx - 1].lower() == "join"):
                # Search through all of the table names in the database we are looking at

                # TODO: Use spacy to tag all table names using named entity recognition (NER).
                for jdx, tableName in enumerate(schemas[iquery['db_id']].tableNames):
                    # If the table names match, indicate it in tablesArray
                    if tableName.lower() == queryToken.lower():
                        tablesArray[jdx] = 1
            else:
                # Search through all of the column names in the database we are looking at
                for jdx, tableName in enumerate(schemas[iquery['db_id']].columnNames):
                    # If the table names match, indicate it in tablesArray
                    if tableName.lower() == queryToken.lower():
                        columnsArray[jdx] = 1
    
        tables.append(tablesArray)
        columns.append(columnsArray)
    
    return np.array(tables), np.array(columns)

Finally, we create the Pandas Dataframe for our training set, then use `train_test_split` to create training and testing data.

e.g.

| x     | y |
|----------|-----|
| "Give me a list of all users over the age of 20"     | [0,1,1,0,0,1]  |
| "Can you give me a list of the student rosters sorted in alphabetical order?"     | [0,1,1,1]  |

In [6]:
trainDF = pd.DataFrame

def loadTrainDF(schemaDoc, queryDoc, schemaName):
 
    tableEmbeddings, columnEmbeddings = getEmbeddings(schemaDoc, queryDoc, schemaName)
    embeddings = np.concatenate((tableEmbeddings, columnEmbeddings),axis=1)

    questions = []
    filteredQueryDoc = [query for query in queryDoc if query['db_id'] == schemaName]

    # Input processing (Question)
    for query in filteredQueryDoc:
        question = query['question']

        # Stem and tokenize each word in the question + remove all stopwords from the question.
        words = tokenizer.tokenize(question)
        processed_words = [stemmer.stem(word) for word in words if word.lower() not in stopWordSet]
        # print(processed_words)
        processed_question = ' '.join(processed_words)
        print(processed_question)
        print(type(processed_question))
        questions.append(processed_question)

    trainDF = pd.DataFrame(data={'question': questions})

    # Output processing (Table labels vector)
    for i in range(embeddings.shape[1]):
        trainDF[i] = embeddings[:,i]

    return trainDF

In [7]:
# Example: college_2
trainDF = loadTrainDF(schemaDoc, queryDoc, 'college_2')
trainDF

find build room capac 50
<class 'str'>
distinct build capac greater 50
<class 'str'>
count number room lamberton build
<class 'str'>
mani classroom lamberton
<class 'str'>
name build depart whose budget averag budget
<class 'str'>
give name build depart greater averag budget
<class 'str'>
find room number room sit 50 100 student build
<class 'str'>
room number correspond build classroom seat 50 100 student
<class 'str'>
find name build depart highest budget
<class 'str'>
depart name correspond build depart greatest budget
<class 'str'>
name student highest total credit histori depart
<class 'str'>
give name student histori depart credit
<class 'str'>
mani room lamberton build
<class 'str'>
count number classroom lamberton
<class 'str'>
mani student advisor
<class 'str'>
count number student advisor
<class 'str'>
mani depart offer cours
<class 'str'>
count number depart offer cours
<class 'str'>
mani differ cours offer physic depart
<class 'str'>
count number cours physic depart
<class 

,question,0,1,2,3,4,5,6,7,8,...,48,49,50,51,52,53,54,55,56,57
0,find build room capac 50,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,distinct build capac greater 50,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,count number room lamberton build,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,mani classroom lamberton,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,name build depart whose budget averag budget,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165,name averag salari depart averag salari higher...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
166,find name instructor salari greater least one ...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
167,name instructor earn least one instructor biol...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
168,find name instructor whose salari greater sala...,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
from sklearn.model_selection import train_test_split

# Split it into train / test subsets
X_train, X_test, y_train, y_test = train_test_split(trainDF['question'], trainDF.drop('question',axis=1), test_size=0.2,
                                                            random_state=42)

# FIXME: Anyway to add a feature similar to 'stratify=trainDF['question']', without breaking?

# Modeling

Prepare our model by loading BERT, and creating the model architecture.

In [9]:
def get_sentence_embeddings(bert_preprocess_model, bert_encoder_model, sentences = []):
    
    text_preprocessed = bert_preprocess_model(sentences)
    return bert_encoder_model(text_preprocessed)['pooled_output']

In [10]:
# Code Adopted and modified from YouTube video: https://www.youtube.com/watch?v=7kLi8u2dJz0&t=719s
preprocess_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'
encoder_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'

In [11]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)
bert_model = hub.KerasLayer(encoder_url)

2023-05-08 19:42:38.588106: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:42:38.588400: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:42:38.588497: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:42:38.588738: I tensorflow/core/

In [12]:
# ! Code Adopted and Modified from https://github.com/eclarson/MachineLearningNotebooks and https://www.youtube.com/watch?v=hOCDJyZ6quA
# Notebook 13: RNN Basics
from tensorflow.keras.layers import Dense, LSTM, Dropout, Input

def build_model(NUM_CLASSES=1):
    # BERT Layers
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name="text")
    preprocessed_text = bert_preprocess_model(text_input)
    outputs = bert_model(preprocessed_text)

    # NN Layers
    l = tf.keras.layers.Dropout(0.2, name="dropout")(outputs['pooled_output'])
    l = tf.keras.layers.Dense(256, activation='sigmoid', name='hidden1')(l)
    l = tf.keras.layers.Dense(128, activation='sigmoid', name='hidden2')(l)
    l = tf.keras.layers.Dense(NUM_CLASSES, activation='sigmoid', name='output')(l)

    # Final Model
    model = tf.keras.Model(inputs=[text_input], outputs=[l])

    METRICS = [
        tf.keras.metrics.BinaryAccuracy(name='accuracy'),
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]

    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=METRICS)

    return model

In [13]:
print(X_train.shape)
print(y_train.shape)

model = build_model(y_train.shape[1])
model.fit(X_train, y_train, epochs=5)
model.save('college_2')

(136,)
(136, 58)


2023-05-08 18:22:32.812531: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-08 18:22:32.824925: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-05-08 18:22:32.905569: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-05-08 1

Epoch 1/5


2023-05-08 18:22:35.072602: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_3' with dtype float and shape [?,?,768]
	 [[{{node gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_3}}]]
2023-05-08 18:22:35.072725: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_4' with dtype float and shape [?,?,768]
	 [[{{node gradients/bert_encoder/StatefulPartitionedCall_grad/bert_encoder/StatefulPartitionedCall_4}}]]
2023

5/5 [==============================] - 14s 2s/step - loss: 0.6316 - accuracy: 0.6296 - precision: 0.1790 - recall: 0.3858
Epoch 2/5
5/5 [==============================] - 10s 2s/step - loss: 0.4956 - accuracy: 0.8155 - precision: 0.2742 - recall: 0.1134
Epoch 3/5
5/5 [==============================] - 10s 2s/step - loss: 0.4335 - accuracy: 0.8372 - precision: 0.2734 - recall: 0.0285
Epoch 4/5
5/5 [==============================] - 10s 2s/step - loss: 0.4012 - accuracy: 0.8446 - precision: 0.0000e+00 - recall: 0.0000e+00
Epoch 5/5
5/5 [==============================] - 10s 2s/step - loss: 0.3847 - accuracy: 0.8446 - precision: 0.0000e+00 - recall: 0.0000e+00


2023-05-08 18:23:26.493025: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-08 18:23:26.506223: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'text' with dtype string and shape [?]
	 [[{{node text}}]]
2023-05-08 18:23:26.548500: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype int32 and shape [?,128]
	 [[{{node inputs}}]]
2023-05-08 18:23:26.548573

INFO:tensorflow:Assets written to: college_2/assets


INFO:tensorflow:Assets written to: college_2/assets


Repeating with the other 3 databases:

In [15]:
trainDF = loadTrainDF(schemaDoc, queryDoc, 'college_2')

# hr_1, store_1, and soccer_2
trainDF_hr = loadTrainDF(schemaDoc, queryDoc, 'hr_1')
X_train, X_test, y_train, y_test = train_test_split(trainDF_hr['question'], trainDF_hr.drop('question',axis=1), test_size=0.2,
                                                            random_state=42)
model_hr = build_model(y_train.shape[1])
model_hr.fit(X_train, y_train, epochs=5)
model_hr.save('hr_1')

trainDF_store = loadTrainDF(schemaDoc, queryDoc, 'store_1')
X_train, X_test, y_train, y_test = train_test_split(trainDF_store['question'], trainDF_store.drop('question',axis=1), test_size=0.2,
                                                            random_state=42)
model_store = build_model(y_train.shape[1])
model_store.fit(X_train, y_train, epochs=5)
model_store.save('store_1')

trainDF_soccer = loadTrainDF(schemaDoc, queryDoc, 'soccer_2')
X_train, X_test, y_train, y_test = train_test_split(trainDF_soccer['question'], trainDF_soccer.drop('question',axis=1), test_size=0.2,
                                                            random_state=42)
model_soccer = build_model(y_train.shape[1])
model_soccer.fit(X_train, y_train, epochs=5)
model_soccer.save('soccer_2')

find build room capac 50
<class 'str'>
distinct build capac greater 50
<class 'str'>
count number room lamberton build
<class 'str'>
mani classroom lamberton
<class 'str'>
name build depart whose budget averag budget
<class 'str'>
give name build depart greater averag budget
<class 'str'>
find room number room sit 50 100 student build
<class 'str'>
room number correspond build classroom seat 50 100 student
<class 'str'>
find name build depart highest budget
<class 'str'>
depart name correspond build depart greatest budget
<class 'str'>
name student highest total credit histori depart
<class 'str'>
give name student histori depart credit
<class 'str'>
mani room lamberton build
<class 'str'>
count number classroom lamberton
<class 'str'>
mani student advisor
<class 'str'>
count number student advisor
<class 'str'>
mani depart offer cours
<class 'str'>
count number depart offer cours
<class 'str'>
mani differ cours offer physic depart
<class 'str'>
count number cours physic depart
<class 

2023-05-08 18:25:42.054739: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-05-08 18:25:42.104271: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [?,128]
	 [[{{node Placeholder_1}}]]
2023-05-08 18:25:42.104332: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32 and shape [?,128]
	 [[{{node

4/4 [==============================] - 9s 2s/step - loss: 0.6266 - accuracy: 0.6758 - precision: 0.2242 - recall: 0.4810
Epoch 2/5
4/4 [==============================] - 7s 2s/step - loss: 0.4825 - accuracy: 0.8071 - precision: 0.3401 - recall: 0.3180
Epoch 3/5
4/4 [==============================] - 8s 2s/step - loss: 0.4126 - accuracy: 0.8520 - precision: 0.5045 - recall: 0.1772
Epoch 4/5
4/4 [==============================] - 8s 2s/step - loss: 0.3723 - accuracy: 0.8673 - precision: 0.8384 - recall: 0.1313
Epoch 5/5
4/4 [==============================] - 8s 2s/step - loss: 0.3484 - accuracy: 0.8670 - precision: 0.8300 - recall: 0.1313


2023-05-08 18:26:22.826135: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'text' with dtype string and shape [?]
	 [[{{node text}}]]
2023-05-08 18:26:22.916021: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-08 18:26:22.929599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'text' with dtype string and shape [?]
	 [[{{node text}}]]
2023-05-08 18:26:22.976454: I ten

INFO:tensorflow:Assets written to: hr_1/assets


...

2023-05-08 18:27:23.927583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder' with dtype string and shape [?]
	 [[{{node Placeholder}}]]
2023-05-08 18:27:23.985517: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_1' with dtype int32 and shape [?,128]
	 [[{{node Placeholder_1}}]]
2023-05-08 18:27:23.985583: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder_2' with dtype int32 and shape [?,128]
	 [[{{node

3/3 [==============================] - 8s 2s/step - loss: 0.6996 - accuracy: 0.5167 - precision: 0.2374 - recall: 0.2919
Epoch 2/5
3/3 [==============================] - 7s 2s/step - loss: 0.5867 - accuracy: 0.6984 - precision: 0.4457 - recall: 0.1108
Epoch 3/5
3/3 [==============================] - 7s 2s/step - loss: 0.5567 - accuracy: 0.7040 - precision: 0.4927 - recall: 0.2730
Epoch 4/5
3/3 [==============================] - 7s 2s/step - loss: 0.5526 - accuracy: 0.7270 - precision: 0.6121 - recall: 0.1919
Epoch 5/5
3/3 [==============================] - 7s 2s/step - loss: 0.5528 - accuracy: 0.7175 - precision: 0.5833 - recall: 0.1324


2023-05-08 18:27:59.992455: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'text' with dtype string and shape [?]
	 [[{{node text}}]]
2023-05-08 18:28:00.082708: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string and shape [?]
	 [[{{node inputs}}]]
2023-05-08 18:28:00.096179: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'text' with dtype string and shape [?]
	 [[{{node text}}]]
2023-05-08 18:28:00.140414: I ten

INFO:tensorflow:Assets written to: soccer_2/assets


INFO:tensorflow:Assets written to: soccer_2/assets


In [14]:
print('Loading College Model...')
college_model = tf.keras.models.load_model('college_2')
print('Loading HR Model...')
hr_model = tf.keras.models.load_model('hr_1')
print('Loading Soccer Model...')
soccer_model = tf.keras.models.load_model('soccer_2')
print('Loading Store Model...')
store_model = tf.keras.models.load_model('store_1')

Loading College Model...


2023-05-08 19:43:42.299908: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:42.300192: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:42.300275: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:42.300510: I tensorflow/core/

Loading HR Model...


2023-05-08 19:43:48.801223: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:48.801507: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:48.801590: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:48.801834: I tensorflow/core/

Loading Soccer Model...


2023-05-08 19:43:55.437517: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:55.437809: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:55.437893: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:43:55.438128: I tensorflow/core/

Loading Store Model...


2023-05-08 19:44:05.724801: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:44:05.725097: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:44:05.725177: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'seq_length' with dtype int32
	 [[{{node seq_length}}]]
2023-05-08 19:44:05.725408: I tensorflow/core/

In [58]:
import random

def randomTest(model, model_name, schemaDoc, queryDoc):
    choice = 1

    tableEmbeddings, columnEmbeddings = getEmbeddings(schemaDoc, queryDoc, model_name)
    embeddings = np.concatenate((tableEmbeddings, columnEmbeddings),axis=1)

    ridx = random.randint(0, embeddings.shape[0])
    target_vect = embeddings[ridx]

    # model_name == db_id
    while choice == 1:

        schemas = {}

        for idx, iSchema in enumerate(schemaDoc):
            dbid = iSchema['db_id']
            schemaObj = Schema(dbid)
            for columnName in iSchema['table_names_original']:
                schemaObj.tableNames.append(columnName)
            for columnName in iSchema['column_names_original']:
                schemaObj.columnNames.append(columnName[1])
                schemaObj.columnAtributes.append(columnName[0])
            #print("Schema", idx, ":", schemaObj.id, ":", schemaObj.tableNames)
            #print("Columns:", schemaObj.columnNames)
            #print("Atributes:", schemaObj.columnAtributes)
            #print()
            schemas[dbid] = schemaObj

        allNames = schemas[model_name].tableNames + schemas[model_name].columnNames
        tablesInQuestion = []
        columnsInQuestion = []

        # print(allNames,len(allNames))
        # print(target_vect,len(target_vect))

        for i in range(len(target_vect)):
            if target_vect[i] == 1:
                if i < len(schemas[model_name].tableNames):
                    tablesInQuestion.append(allNames[i])
                else:
                    columnsInQuestion.append(allNames[i])

        question = input('Enter a question for DB '+model_name+' with the following characteristics: \nTables: '+str(tablesInQuestion)+'; Columns: '+str(columnsInQuestion))

        # [1, 0, 0, 0, 1, 0, 0, 0, 0]
        # Try to mention table name 'College' and attribute name 'College Name'
        # Question:
        # Model would take in question and then predict the vector
        # If the generated vector matches (or is really close to matching) the initial vector then our model is working
        # Otherwise it's not

        # 
        predictions = model.predict([question])

        predictionVect = []
        for prediction in predictions[0]:
            if prediction > 0.5:
                predictionVect.append(1)
            else:
                predictionVect.append(0)

        print(predictions)
        print(predictionVect)

        choice = int(input('Repeat (1) or new random test (0)'))


while True:
    randomTest(college_model, 'college_2', schemaDoc, queryDoc)

1/1 [==============================] - 0s 97ms/step
[[0.08319402 0.06016282 0.26973224 0.3398303  0.17093569 0.10043813
  0.17845702 0.04544022 0.08948554 0.11187072 0.11978232 0.2140223
  0.11549921 0.08607455 0.06944267 0.484198   0.15935041 0.13214
  0.3264887  0.1961458  0.46838057 0.15763868 0.25402895 0.32034606
  0.43385023 0.13329075 0.26867706 0.03388811 0.22064337 0.17086983
  0.15807605 0.10523134 0.08667213 0.2503022  0.28982335 0.04828289
  0.10785684 0.17070688 0.21443592 0.3396823  0.45065486 0.04730571
  0.21878389 0.2684123  0.04943829 0.11645213 0.17318702 0.1003095
  0.07166553 0.1192168  0.10327911 0.03641745 0.05834133 0.08951091
  0.07340765 0.07879789 0.2651309  0.11329845]]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


ValueError: invalid literal for int() with base 10: ''